In [13]:
import pandas as pd
import numpy as np
from openai import OpenAI
import joblib
from tqdm.notebook import tqdm
from glob import glob
import random
import json
import os
from os import path, listdir

In [64]:
path_dir = '/home/crazyjeannot/BAMMAN_CHAPITRES/'
path_entities = '/home/crazyjeannot/BAMMAN_CHAPITRES/*.entities'
path_book = '/home/crazyjeannot/BAMMAN_CHAPITRES/*.book'
path_output = '/home/crazyjeannot/Documents/doctorat/JCLS_paper/data/ALL/'

In [53]:
def get_characterization(booknlp_data, N=10):
    list_main = []
    for i in range(min(len(booknlp_data["characters"]), N)):
        list_main.extend([item['w'] for item in booknlp_data["characters"][i]['agent']])
        list_main.extend([item['w'] for item in booknlp_data["characters"][i]['patient']])
        list_main.extend([item['w'] for item in booknlp_data["characters"][i]['mod']])
    return list_main

In [62]:
def get_booknlp(path_entities, path_book, path_output, N=10):
    list_str_res = []
    categories_to_filter = ['GPE', 'LOC', 'VEH', 'TIME', 'FAC']

    entities_files = sorted(glob(path_entities))
    book_files = sorted(glob(path_book))
    
    for doc_entities, doc_book in tqdm(zip(entities_files, book_files), total=2944):
        file_entities = path.splitext(path.basename(doc_entities))[0]
        file_book = path.splitext(path.basename(doc_book))[0]
        
        if  file_entities == file_book:
        
            # GET ENTITIES / CHRONOTOPE
            df_entities = pd.read_csv(doc_entities, sep='\t', engine="python", on_bad_lines='skip')#, encoding_errors='replace')#lineterminator='\n',         
            df_chronotope = df_entities[df_entities['cat'].isin(categories_to_filter)]
            chronotope = list(df_chronotope.text)

            # GET CHARACTERIZATION
            with open(doc_book, "r") as booknlp_book:
                book_data = json.load(booknlp_book)
                characterization = get_characterization(book_data, N)

        else: 
            print(doc_entities)
            print(doc_book)
            print("NOT THE SAME FILE")
            break
            
        characterization.extend(chronotope)
        random.shuffle(chronotope)
        booknlp_str = ' '.join(chronotope)
        list_str_res.append(booknlp_str)
        
        # Écriture fichier booknlp
        if not os.path.exists(path_output):
            os.makedirs(path_output)
        with open(path_output+file_book+'.txt', 'w') as file_to_write:
            file_to_write.write(booknlp_str)

    return list_str_res

In [63]:
list_str_res = get_booknlp(path_entities, path_book, path_output_TIME)

  0%|          | 0/2944 [00:00<?, ?it/s]